## Prepare Base Model

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection'

In [5]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_dropout_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from plantDiseaseDetection.constants import *
from plantDiseaseDetection.utils import read_yaml, create_directories

In [7]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:

        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_dropout_rate = self.params.DROPOUT_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config


In [8]:
import tensorflow as tf

2023-11-10 22:28:41.341461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 22:28:41.385558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 22:28:41.385622: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 22:28:41.385672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 22:28:41.396423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 22:28:41.397571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
# Prepare base model components

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def _get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        self._save_model(path=self.config.base_model_path, model=self.model)

    def _save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    def _prepare_full_model(self, model: tf.keras.Model, classes: list, freeze_all: bool, freeze_till, learning_rate: float, dropout_rate: float=0.5):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[freeze_till]:
                layer.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        dense_layer = tf.keras.layers.Dense(256, activation='relu')(flatten_in)
        dropout_layer = tf.keras.layers.Dropout(dropout_rate)(dense_layer)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(dropout_layer)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

        full_model.summary()

        return full_model
    
    def _update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        
        self._save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    def initiate_prepare_base_model(self):
        self._get_base_model()
        self._update_base_model()
    

In [12]:
# Pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.initiate_prepare_base_model()
except Exception as e:
    raise e

2023-11-10 23:50:55,652 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: config/config.yaml loaded successfully
2023-11-10 23:50:55,656 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: params.yaml loaded successfully
2023-11-10 23:50:55,658 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts
2023-11-10 23:50:55,659 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts/prepare_base_model
2023-11-10 23:50:56,077 : tensorflow - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   